In [41]:
import sys, math, time
import re
import os
import pandas as pd
import numpy as np
import math
from array import array

np.set_printoptions(suppress=True)

In [2]:
# PHYSICAL CONSTANTS

GAS_CONSTANT = 8.31446261815324 # J / (mol·K) #
PLANCK_CONSTANT = 6.62607015e-34 # m2 kg / s #
BOLTZMANN_CONSTANT = 1.3806488e-23 # m2·kg / (s2⋅K) or J / K # 
SPEED_OF_LIGHT = 2.99792458e10 # m / s #
AVOGADRO_CONSTANT = 6.02214076e23 # / mol #
AMU_to_KG = 1.66053886E-27 # kg #
autokcal = 627.509541
kcaltokj = 4.184
jtohartree = 2.2937104486906E+17
PI = math.pi
k = 3.166811563E-6 #Boltzmann Constant in atomic units

#new values#
protoelec = 1836.152669389233
k_real = BOLTZMANN_CONSTANT*jtohartree #approx same as below#

atom_map = {'H': 1.008,	'He': 4.0026,	'Li': 7,	'Be': 9.012183,	'B': 10.81,	'C': 12.011,	'N': 14.007,	'O': 15.999,	'F': 18.99840316,	'Ne': 20.18,	'Na': 22.9897693,	'Mg': 24.305,	'Al': 26.981538,	'Si': 28.085,	'P': 30.973762,	'S': 32.07,	'Cl': 35.45,	'Ar': 39.9,	'K': 39.0983,	'Ca': 40.08,	'Sc': 44.95591,	'Ti': 47.867,	'V': 50.9415,	'Cr': 51.996,	'Mn': 54.93804,	'Fe': 55.84,	'Co': 58.93319,	'Ni': 58.693,	'Cu': 63.55,	'Zn': 65.4,	'Ga': 69.723,	'Ge': 72.63,	'As': 74.92159,	'Se': 78.97,	'Br': 79.9,	'Kr': 83.8,	'Rb': 85.468,	'Sr': 87.62,	'Y': 88.90584,	'Zr': 91.22,	'Nb': 92.90637,	'Mo': 95.95,	'Tc': 96.90636,	'Ru': 101.1,	'Rh': 102.9055,	'Pd': 106.42,	'Ag': 107.868,	'Cd': 112.41,	'In': 114.818,	'Sn': 118.71,	'Sb': 121.76,	'Te': 127.6,	'I': 126.9045,	'Xe': 131.29,	'Cs': 132.905452,	'Ba': 137.33,	'La': 138.9055,	'Ce': 140.116,	'Pr': 140.90766,	'Nd': 144.24,	'Pm': 144.91276,	'Sm': 150.4,	'Eu': 151.964,	'Gd': 157.2,	'Tb': 158.92535,	'Dy': 162.5,	'Ho': 164.93033,	'Er': 167.26,	'Tm': 168.93422,	'Yb': 173.05,	'Lu': 174.9668,	'Hf': 178.49,	'Ta': 180.9479,	'W': 183.84,	'Re': 186.207,	'Os': 190.2,	'Ir': 192.22,	'Pt': 195.08,	'Au': 196.96657,	'Hg': 200.59,	'Tl': 204.383,	'Pb': 207,	'Bi': 208.9804,	'Po': 208.98243,	'At': 209.98715,	'Rn': 222.01758,	'Fr': 223.01973,	'Ra': 226.02541,	'Ac': 227.02775,	'Th': 232.038,	'Pa': 231.03588,	'U': 238.0289,	'Np': 237.048172,	'Pu': 244.0642,	'Am': 243.06138,	'Cm': 247.07035,	'Bk': 247.07031,	'Cf': 251.07959,	'Es': 252.083,	'Fm': 257.09511,	'Md': 258.09843,	'No': 259.101,	'Lr': 266.12,	'Rf': 267.122,	'Db': 268.126,	'Sg': 269.128,	'Bh': 270.133,	'Hs': 269.1336,	'Mt': 277.154,	'Ds': 282.166,	'Rg': 282.169,	'Cn': 286.179,	'Nh': 286.182,	'Fl': 290.192,	'Mc': 290.196,	'Lv': 293.205,	'Ts': 294.211,	'Og': 295.216}

In [41]:
#Variables to add#

NAME = 'TS-BHCl2Sub'
TEMPERATURE = 298.15

In [48]:
#Set variables to use later

DIR_NAME = os.path.join('BCl_Ionic', 'vib', NAME, NAME)
 
SM_SPC = DIR_NAME + '_IRC_B_SPC.out'
P_SPC = DIR_NAME + '_IRC_F_SPC.out'
TS_SPC = DIR_NAME + '_SPC.out'
SM = DIR_NAME + '_IRC_B_T.out'
P = DIR_NAME + '_IRC_F_T.out'
TS = DIR_NAME + '_T.out'
TS_VIB = DIR_NAME + '_T.out'
TS_XYZ = DIR_NAME + '.xyz'

In [49]:
#10-point Gauss-Legendre Quadrature abscissa and weight (exact solution for up to 21st order polynomial)
x = array('d',[-0.9739065285,-0.8650633667,-0.6794095683,-0.4333953941,-0.1488743390,0.1488743390,0.4333953941,0.6794095683,0.8650633667,0.9739065285])
w = array('d',[0.0666713443,0.1494513492,0.2190863625,0.2692667193,0.2955242247,0.2955242247,0.2692667193,0.2190863625,0.1494513492,0.0666713443])

In [50]:
#Create functions

#Read ORCA output for the Final SCF Energy
def final_scf_energy(file):
    with open(file, 'r') as ORCA_output:
        for line in ORCA_output:
            if 'FINAL SINGLE POINT ENERGY' in line:   
                spc = line.strip().split()[-1]
                return float(spc)
    return None

#Read ORCA output for the ZPE
def zero_point_energy(file):
    start_reading = False
    with open(file, 'r') as ORCA_output:
        
        for line in ORCA_output:
        # Check for the start condition
            if line.startswith('THERMOCHEMISTRY AT ' + str(TEMPERATURE) + 'K'):
                start_reading = True
                continue
            
            if start_reading:
                if 'Zero point energy' in line:   
                    ZPE = line.strip().split()[4]
                    return float(ZPE)
        return None

#Read ORCA output for thermal correction to Gibbs Free Energy
def g_corr(file):
    start_reading = False
    with open(file, 'r') as ORCA_output:
        
        for line in ORCA_output:
        # Check for the start condition
            if line.startswith('THERMOCHEMISTRY AT ' + str(TEMPERATURE) + 'K'):
                start_reading = True
                continue
            
            if start_reading:
                if 'G-E(el)' in line:   
                    gcorr = line.strip().split()[2]
                    return float(gcorr)
        return None

#Read ORCA .out for vibrational freq of normal mode of transition
def vibration(file):
    with open(file, 'r') as ORCA_output:
        for line in ORCA_output:
            if '***imaginary mode***' in line:
                novib = line.strip().split()[0].replace(':', '')
                vib = line.strip().split()[1]
                return int(novib), float(vib)
    return None

def AtomMass(file):
    with open(file) as f:
        atoms = []
        lines = f.read().splitlines()
        for i, line in enumerate(lines[2:], start=2):
            split_line = line.split()
            mass = atom_map[split_line[0]]
            atoms.append((split_line[0], mass))
    return atoms

#Calculate the force constant using calculated reduced mass similar to Gaussian
def red_mass_fc(file1,file2):
    atoms = AtomMass(file1)
    novib, vib = vibration(file2)
    novib_row = novib // 6
    masses = [item[1] for item in atoms]
    n_atoms = len(atoms)
    n_rows = n_atoms * 3
    normsq_list = []
    
    with open(file2, 'r') as ORCA_output:
        lines = ORCA_output.readlines()
        
    for i, line in enumerate(lines):
        if line.startswith('NORMAL MODES'):
            for line in lines[i + 8 + (n_rows * novib_row) + novib_row :i + 8 + (n_rows * novib_row) + novib_row + n_rows]:
                data = line.split()[1:]
                disp = float(data[novib%len(data)])
                value = disp**2
                normsq_list.append(value)
    
    red_mass_list = normsq_list.copy()
    for j in range(n_rows):
        red_mass_list[j] = normsq_list[j] / masses[j // 3]
    
    red_mass = 1 / sum(red_mass_list)
    fc = (red_mass * AMU_to_KG * 4 * PI**2 * SPEED_OF_LIGHT**2 * vib**2)/ 100
    
    return red_mass, fc

#Calculate the force constant setting the reduced mass equal to 1
def fc_redmass1(file):
    red_mass = 1.008
    vibno, vib = vibration(file)
    fc = (red_mass * AMU_to_KG * 4 * PI**2 * SPEED_OF_LIGHT**2 * float(vib)**2)/ 100 
    return red_mass, fc

In [51]:
#Parameters B, ALPHA, a, b, d of Eckart Potential
def Bee(V_max,V_r, V_p):
    bee = (V_max ** 0.5 + (V_max - (V_p - V_r)) ** 0.5) ** 2
    return bee

def ALPHA(B,F_s,V_max,V_r, V_p):
    alpha = (B * F_s / (2 * V_max * (V_max - (V_p - V_r)))) ** 0.5
    return alpha

def A(E,mu,ALPHA):
    a =  2 * PI * (2 * mu * E)**0.5 / ALPHA
    return a

def B(E,mu,V_p,V_r,ALPHA):
    b =  2 * PI * (2 * mu * (E - (V_p - V_r)))**0.5 / ALPHA
    return b

def D(bee,mu,ALPHA):
    d =  2 * PI * abs((2 * mu * bee - (ALPHA/2)**2))**0.5 / ALPHA
    return d

#Calculation of Transmission Probabilty of Eckart Potential
def T(a,b,d):
    T = (math.cosh(a+b) - math.cosh(a-b))/(math.cosh(a+b) + math.cosh(d))
    return T

#Calculation of SINH function of Kappa
def S(Vmax,E):
    S = math.sinh(((Vmax-E)) / (TEMPERATURE*k))
    return S

In [52]:
#Think about scaling#

#Input output files to extract data
V_r = float(final_scf_energy(SM_SPC)) + float(zero_point_energy(SM))
V_p = float(final_scf_energy(P_SPC)) + float(zero_point_energy(P))
V_max = (float(final_scf_energy(TS_SPC)) + float(zero_point_energy(TS)))
G_r = float(final_scf_energy(SM_SPC)) + float(g_corr(SM)) + 0.00398041762
G_p = float(final_scf_energy(P_SPC)) + float(g_corr(P)) + 0.00398041762
G_ts = float(final_scf_energy(TS_SPC)) + float(g_corr(TS)) + 0.00398041762

if V_r > V_p:
    E_o = V_r
else:
    E_o = V_p

V_max = V_max - V_r
V_p = V_p - V_r
E_o = E_o - V_r
V_r = V_r - V_r
        
y = (V_max - E_o)/2.0 
z = (V_max + E_o)/2.0 

In [53]:
# Specifing Parameters for the Eckart Potential - input .vib file

mu_uncrr, F_s_uncrr = red_mass_fc(TS_XYZ,TS_VIB)
FQn, FQ_uncrr = vibration(TS)

mu = float(mu_uncrr)*1836 #convert to electron mass#
F_s = float(F_s_uncrr)/15.569141 #convert to Hartree per bohr^2#
FQ = abs(float(FQ_uncrr))*4.55633e-6 #convert to Hartree#

bee = Bee(V_max,V_r,V_p)
alpha = ALPHA(bee,F_s,V_max,V_r,V_p)
d = D(bee,mu,alpha)

In [54]:
#Calculation of Uncorrected Gibbs Free Energy Barrier in kcal/mol and Rate
delta_g_dagg = (G_ts - G_r)*2625.5
delta_g_daggk = (G_ts - G_r)*autokcal
ln_uncorr_rate = math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT))-delta_g_dagg*1000/(GAS_CONSTANT*TEMPERATURE)  
uncorr_rate = "{:.10E}".format(math.exp(ln_uncorr_rate))

#Calculation of Wigner tunneling correction
kappa_w = 1+(( FQ / (k*TEMPERATURE) )**2) / 24

#Calculation of Skodje tunneling correction
alpha_s = (2*PI/(FQ))
beta_s = (1/ (k*TEMPERATURE))

if V_p < V_r:
    Vee = 0
else:
    Vee = V_p-V_r
if alpha_s > beta_s:
    kappa_s = (beta_s*PI/alpha_s)/(math.sin(beta_s*PI/alpha_s))+(beta_s/(beta_s-alpha_s)*math.exp((beta_s-alpha_s)*(V_max-Vee)))
elif alpha_s < beta_s:
    kappa_s = (beta_s/(beta_s-alpha_s)*(math.exp((beta_s-alpha_s)*(V_max-Vee))-1))
else:
    kappa_s = alpha_s*(V_max-Vee)
       
#Calculation of Eckart tunneling correction using 10-point Gauss-Legendre Quadrature
kappa = 1
for i in range(0,10):
    a = A((x[i] * y + z),mu,alpha)
    b = B((x[i] * y + z),mu,V_p,V_r,alpha)
    kappa = (2 * y  / (TEMPERATURE*k) * w[i] * S((V_max),(x[i] * y + z)) * T(a,b,d)) + kappa

#Calculation of Wigner Apparent Gibbs Free Energy Barrier in kcal/mol and Rate
corr_rate_w = "{:.10E}".format(kappa_w*math.exp(ln_uncorr_rate))

delta_g_dagg_app_w = GAS_CONSTANT * TEMPERATURE * (math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT)) - math.log(kappa_w) - ln_uncorr_rate) /1000
delta_g_dagg_app_wk = delta_g_dagg_app_w / kcaltokj
w_cor = delta_g_dagg_app_wk - delta_g_daggk  

#Calculation of Skodje Apparent Gibbs Free Energy Barrier in kcal/mol and Rate
corr_rate_s = "{:.10E}".format(kappa_s*math.exp(ln_uncorr_rate))

delta_g_dagg_app_s = GAS_CONSTANT * TEMPERATURE * (math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT)) - math.log(kappa_s) - ln_uncorr_rate) /1000
delta_g_dagg_app_sk = delta_g_dagg_app_s / kcaltokj
s_cor = delta_g_dagg_app_sk - delta_g_daggk

#Calculation of Eckart Apparent Gibbs Free Energy Barrier in kJ/mol and Rate
corr_rate = "{:.10E}".format(kappa*math.exp(ln_uncorr_rate))

delta_g_dagg_app = GAS_CONSTANT * TEMPERATURE * (math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT)) - math.log(kappa) - ln_uncorr_rate) /1000
delta_g_dagg_appk = delta_g_dagg_app / kcaltokj
e_cor = delta_g_dagg_appk - delta_g_daggk

In [55]:
#Print section

print(FQ_uncrr)

data = []
data.append([NAME, kappa_w, kappa_s, kappa, uncorr_rate, corr_rate_w, corr_rate_s, corr_rate, delta_g_daggk,delta_g_dagg_app_wk,delta_g_dagg_app_sk,delta_g_dagg_appk])

df = pd.DataFrame(data, columns=['Name', 'Kw', 'Ksw', 'Ke','k','kw','ksw','ke','G*','G*w','G*sw','G*e'])

pd.set_option('display.float_format', '{:.2f}'.format)

df

-141.02


,Name,Kw,Ksw,Ke,k,kw,ksw,ke,G*,G*w,G*sw,G*e
0,TS-BHCl2Sub,1.02,0.94,1.00,4.8886110980E+12,4.9829415551E+12,4.6187914884E+12,4.8932556007E+12,0.14,0.13,0.18,0.14


In [56]:
#Rate calculator

activation = (16 / autokcal) * 2625.5

ln_act_rate = math.log(BOLTZMANN_CONSTANT * TEMPERATURE / (PLANCK_CONSTANT))-activation*1000/(GAS_CONSTANT*TEMPERATURE)  
rate = "{:.10E}".format(math.exp(ln_act_rate))
print(rate)

1.1619307847E+01


In [98]:
adjrate = "{:.10E}".format(math.exp(ln_act_rate)*2)
print(adjrate)

2.3238615693E+01


In [70]:
kappa_w

1.023353533638133

In [71]:
kappa_s

-0.4372748723052966

In [72]:
FQ_uncrr

-155.14

In [1]:
import numpy as np

In [ ]:
q = [[-0.009345915, -0.02097390988037, -0.67467928801347], [-0.009345915, 0.73318230008236, -0.076387718777182], [-0.009345915, -0.77513012920199, -0.07638771620935]]
S = [[0, 0, 0], [0.962163214294, 0, 0], [0.962163188253, -0.0898888039, -1.003853289]]

q_array = np.array(q)
S_array = np.array(S)

In [20]:
Dt, residuals, rank, s = np.linalg.lstsq(q_array.T, S_array.T)
D = Dt
D

array([[ 1.25436445e-03,  2.03846116e-02, -1.66443409e-02],
       [-1.39692783e-01, -1.35413007e+00,  9.96563059e-01],
       [-1.95677780e-02, -1.23400506e+00,  1.11668806e+00]])

In [21]:
S = D @ q_array
S

array([[ 0.        ,  0.02770854, -0.00176713],
       [ 0.        , -1.75900923,  0.16878808],
       [ 0.        , -1.75900923,  0.05557553]])

In [22]:
q = np.linalg.inv(D) @ S_array
q

array([[ 1.35278373e+17,  8.54039673e+16, -7.24258441e+16],
       [-6.53895828e+16, -4.12817633e+16,  3.50085207e+16],
       [-6.98887912e+16, -4.41222044e+16,  3.74173238e+16]])

In [25]:
np.linalg.inv(S_array)

array([[ 1.45119901e+09, -7.01466876e+08, -7.49732142e+08],
       [-2.29867357e+09,  1.11111112e+09,  1.18756245e+09],
       [ 9.26349214e-01, -0.00000000e+00, -9.26349214e-01]])

In [28]:
D_inv = q_array @ np.linalg.inv(S_array)
D = np.linalg.inv(D_inv)
D

array([[-6.97628040e+08, -5.49464189e+06, -5.49464189e+06],
       [-6.97627886e+08, -5.49464189e+06, -5.49464189e+06],
       [-6.97628184e+08, -5.49464189e+06, -5.49464189e+06]])

In [29]:
S = D @ q_array
S

array([[ 0.00000000e+00, -1.60350606e-10, -3.50178220e-01],
       [ 0.00000000e+00,  7.70971969e-10,  8.03605016e-01],
       [ 0.00000000e+00, -1.60350606e-10, -1.42968472e+00]])

In [30]:
 hess = np.array([[  5.69133000e-01,  -7.00000000e-06,  -0.00000000e+00,
          -3.41891000e-01,   9.00000000e-06,   0.00000000e+00,
          -7.56840000e-02,   9.41130000e-02,   0.00000000e+00,
          -7.56850000e-02,  -4.69810000e-02,  -8.13740000e-02,
          -7.56850000e-02,  -4.69810000e-02,   8.13740000e-02],
        [ -4.00000000e-06,   5.69121000e-01,  -0.00000000e+00,
           3.00000000e-06,  -4.24950000e-02,  -0.00000000e+00,
           9.39440000e-02,  -3.08614000e-01,  -0.00000000e+00,
          -4.69710000e-02,  -1.08902000e-01,  -1.15083000e-01,
          -4.69710000e-02,  -1.08902000e-01,   1.15083000e-01],
        [ -0.00000000e+00,   0.00000000e+00,   5.69119000e-01,
           0.00000000e+00,   0.00000000e+00,  -4.24940000e-02,
          -0.00000000e+00,  -0.00000000e+00,  -4.24910000e-02,
          -8.13530000e-02,  -1.15066000e-01,  -2.41795000e-01,
           8.13530000e-02,   1.15066000e-01,  -2.41795000e-01],
        [ -3.41984000e-01,  -1.00000000e-06,   0.00000000e+00,
           3.64262000e-01,  -6.00000000e-06,   0.00000000e+00,
          -7.49300000e-03,  -1.77600000e-03,  -0.00000000e+00,
          -7.49300000e-03,   8.13000000e-04,   1.40700000e-03,
          -7.49300000e-03,   8.13000000e-04,  -1.40700000e-03],
        [  4.00000000e-06,  -4.24390000e-02,  -0.00000000e+00,
          -2.00000000e-06,   4.11320000e-02,   0.00000000e+00,
           3.07950000e-02,   2.80800000e-03,  -0.00000000e+00,
          -1.53980000e-02,  -6.49000000e-04,   2.00500000e-03,
          -1.53980000e-02,  -6.49000000e-04,  -2.00500000e-03],
        [  0.00000000e+00,  -0.00000000e+00,  -4.24330000e-02,
          -0.00000000e+00,  -0.00000000e+00,   4.11270000e-02,
           0.00000000e+00,   0.00000000e+00,  -1.84200000e-03,
          -2.66720000e-02,   2.00500000e-03,   1.66600000e-03,
           2.66720000e-02,  -2.00500000e-03,   1.66600000e-03],
        [ -7.57160000e-02,   9.41410000e-02,  -0.00000000e+00,
          -7.45800000e-03,   3.07520000e-02,   0.00000000e+00,
           7.68220000e-02,  -1.01620000e-01,  -0.00000000e+00,
           3.17800000e-03,  -1.16250000e-02,   1.42000000e-03,
           3.17800000e-03,  -1.16250000e-02,  -1.42000000e-03],
        [  9.41300000e-02,  -3.08688000e-01,   0.00000000e+00,
          -1.77800000e-03,   2.75100000e-03,  -0.00000000e+00,
          -1.01487000e-01,   3.28330000e-01,  -0.00000000e+00,
           4.57900000e-03,  -1.13170000e-02,   1.99700000e-03,
           4.57900000e-03,  -1.13180000e-02,  -1.99700000e-03],
        [ -0.00000000e+00,   0.00000000e+00,  -4.24370000e-02,
          -0.00000000e+00,  -0.00000000e+00,  -1.84100000e-03,
           0.00000000e+00,   0.00000000e+00,   4.11280000e-02,
           1.07790000e-02,  -2.44790000e-02,   1.66700000e-03,
          -1.07790000e-02,   2.44790000e-02,   1.66700000e-03],
        [ -7.57170000e-02,  -4.70670000e-02,  -8.15280000e-02,
          -7.45600000e-03,  -1.53770000e-02,  -2.66320000e-02,
           3.17700000e-03,   4.64200000e-03,   1.07500000e-02,
           7.68200000e-02,   5.07480000e-02,   8.79090000e-02,
           3.17900000e-03,   7.04600000e-03,   9.36100000e-03],
        [ -4.70650000e-02,  -1.08997000e-01,  -1.15292000e-01,
           8.89000000e-04,  -6.94000000e-04,   1.98700000e-03,
          -1.16260000e-02,  -1.12620000e-02,  -2.44470000e-02,
           5.07430000e-02,   1.12710000e-01,   1.24317000e-01,
           7.04700000e-03,   8.15900000e-03,   1.32360000e-02],
        [ -8.15190000e-02,  -1.15289000e-01,  -2.42125000e-01,
           1.53900000e-03,   1.98700000e-03,   1.60400000e-03,
           1.42200000e-03,   2.11700000e-03,   1.60300000e-03,
           8.78910000e-02,   1.24307000e-01,   2.56265000e-01,
          -9.35400000e-03,  -1.32340000e-02,  -1.78040000e-02],
        [ -7.57170000e-02,  -4.70670000e-02,   8.15290000e-02,
          -7.45600000e-03,  -1.53780000e-02,   2.66320000e-02,
           3.17800000e-03,   4.64200000e-03,  -1.07500000e-02,
           3.17900000e-03,   7.04600000e-03,  -9.36100000e-03,
           7.68200000e-02,   5.07480000e-02,  -8.79090000e-02],
        [ -4.70650000e-02,  -1.08997000e-01,   1.15292000e-01,
           8.89000000e-04,  -6.94000000e-04,  -1.98700000e-03,
          -1.16260000e-02,  -1.12620000e-02,   2.44470000e-02,
           7.04700000e-03,   8.15900000e-03,  -1.32360000e-02,
           5.07430000e-02,   1.12710000e-01,  -1.24316000e-01],
        [  8.15190000e-02,   1.15290000e-01,  -2.42125000e-01,
          -1.53900000e-03,  -1.98700000e-03,   1.60400000e-03,
          -1.42200000e-03,  -2.11700000e-03,   1.60200000e-03,
           9.35400000e-03,   1.32340000e-02,  -1.78040000e-02,
          -8.78910000e-02,  -1.24307000e-01,   2.56265000e-01]])

In [105]:
    geom = np.array([[-0.000000, 0.000000, 0.000000],
                            [2.059801,  0.,  0.],
                            [-0.6866  ,  1.942   ,  0.],
                            [-0.6866  , -0.971   , -1.681821],
                            [-0.6866  , -0.971   ,  1.681821]
                            ])

In [111]:
masses = np.array([12.011, 1.008, 1.008, 1.008, 1.008])

In [76]:
mwh_eigvecs = np.array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -3.94942498e-01,  -3.37783511e-02,   2.26733150e-04,
          -1.01615700e-05,   2.38076000e-06,  -5.69347300e-04,
           7.55487600e-05,   1.51161154e-01,   2.67121574e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -3.37381468e-02,   3.94929999e-01,   4.61315230e-04,
          -5.29110350e-04,  -9.81910000e-07,  -6.43013690e-04,
           2.52277620e-04,   2.67162589e-01,  -1.51128355e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -2.65229270e-04,   4.40318340e-04,  -3.96367433e-01,
          -2.16370000e-07,  -2.18289900e-05,   4.37600000e-08,
          -3.06948024e-01,   2.56769930e-04,  -5.85181800e-05],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -9.05015412e-02,  -7.74361372e-03,   5.19772600e-05,
           5.23018800e-05,   5.71510000e-07,  -4.98496537e-01,
          -2.03810570e-04,  -4.07032056e-01,  -7.17456917e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           4.75409737e-02,  -5.55800882e-01,  -6.49630810e-04,
           5.00779591e-01,   1.79653180e-04,  -9.57941000e-06,
           1.21494800e-05,   1.27263808e-02,  -7.17497870e-03],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           3.70775910e-04,  -6.21831770e-04,   5.58550833e-01,
           1.78587630e-04,  -4.99974958e-01,   1.33660000e-07,
          -1.46393704e-02,   1.22860200e-05,  -2.88351000e-06],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           5.01974298e-01,  -1.61242446e-01,  -5.15354590e-04,
           4.71703108e-01,   1.65649790e-04,   1.66288423e-01,
           1.97720920e-04,   1.91113688e-01,  -1.97933423e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           2.01644306e-01,   3.62669730e-02,  -9.42581400e-05,
           1.66581824e-01,   5.83765200e-05,  -4.70357315e-01,
          -5.37587960e-04,  -5.07599157e-01,   5.88968439e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           3.76458880e-04,  -6.19336180e-04,   5.58505671e-01,
          -1.77989630e-04,   5.00025104e-01,   4.11280000e-07,
          -1.46518096e-02,   1.22816900e-05,  -3.04691000e-06],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           4.75800519e-01,   1.42990392e-01,  -1.76927029e-01,
          -2.36005830e-01,   4.08155849e-01,   1.67086585e-01,
           2.28106479e-01,  -1.53129034e-01,  -3.30090441e-03],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -6.65308974e-02,  -4.21587122e-01,  -2.50371783e-01,
          -3.32664681e-01,  -2.88799673e-01,   2.36292821e-01,
           3.22609804e-01,  -2.13944111e-01,  -2.99942945e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -1.97389599e-01,   2.34254086e-01,   1.25976062e-01,
           2.88303412e-01,   1.15834320e-04,   4.09333053e-01,
           5.44103597e-01,  -3.92668431e-01,  -3.93914218e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           4.76032223e-01,   1.42595457e-01,   1.76607744e-01,
          -2.35714503e-01,  -4.08330289e-01,   1.67086865e-01,
          -2.28361176e-01,  -1.52747122e-01,  -3.38807898e-03],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          -6.61933754e-02,  -4.22141321e-01,   2.49523254e-01,
          -3.32870292e-01,   2.88565032e-01,   2.36293698e-01,
          -3.22955204e-01,  -2.13404013e-01,  -3.01178611e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           1.97557911e-01,  -2.34532857e-01,   1.25191679e-01,
          -2.88303263e-01,  -9.06289600e-05,  -4.09333749e-01,
           5.44744177e-01,   3.91757517e-01,   3.95993516e-02]])

In [37]:
mass_list = []
for mass in masses:
    mass_list.extend([mass]*3)

mass_array = np.array(mass_list)
mass_array

array([12.011, 12.011, 12.011,  1.008,  1.008,  1.008,  1.008,  1.008,
        1.008,  1.008,  1.008,  1.008,  1.008,  1.008,  1.008])

In [103]:
mass_matrix = np.diag(1/np.sqrt(mass_array))
           
hess_mw = mass_matrix @ hess @ mass_matrix
hess_mw

array([[ 0.04738431, -0.00000058,  0.        , -0.09825798,  0.00000259,
         0.        , -0.02175125,  0.02704766,  0.        , -0.02175154,
        -0.01350213, -0.02338653, -0.02175154, -0.01350213,  0.02338653],
       [-0.00000033,  0.04738332,  0.        ,  0.00000086, -0.01221288,
         0.        ,  0.02699909, -0.08869431,  0.        , -0.01349926,
        -0.03129796, -0.03307435, -0.01349926, -0.03129796,  0.03307435],
       [ 0.        ,  0.        ,  0.04738315,  0.        ,  0.        ,
        -0.01221259,  0.        ,  0.        , -0.01221173, -0.0233805 ,
        -0.03306946, -0.06949082,  0.0233805 ,  0.03306946, -0.06949082],
       [-0.0982847 , -0.00000029,  0.        ,  0.36137103, -0.00000595,
         0.        , -0.00743353, -0.0017619 ,  0.        , -0.00743353,
         0.00080655,  0.00139583, -0.00743353,  0.00080655, -0.00139583],
       [ 0.00000115, -0.01219678,  0.        , -0.00000198,  0.04080556,
         0.        ,  0.0305506 ,  0.00278571, 

In [74]:
### this is just the mass weighted hessian ###
eig = np.linalg.eigvalsh(hess_mw) * 4.359744650E-18 / (1.660539040E-27 * 0.529177249E-10 **2)
frequencies = np.sqrt(np.maximum(eig, 0) / (4 * (math.pi ** 2) * (2.99792458e+10 ** 2)))
frequencies

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        , 1292.22553902, 1292.41866597,
       1292.55366488, 1524.37960409, 1524.4862033 , 3100.01978772,
       3239.33923494, 3240.62699567, 3241.12820542])

In [75]:
eigval, eigvect = np.linalg.eigh(hess_mw)
eigval

array([-0.00298288, -0.00293001, -0.00290316, -0.00004975, -0.00001943,
       -0.00000091,  0.06319288,  0.06321177,  0.06322498,  0.08793824,
        0.08795054,  0.36368086,  0.3971041 ,  0.39741989,  0.39754283])

In [77]:
hess_int = mwh_eigvecs.T @ hess_mw @ mwh_eigvecs
hess_int

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , 

In [78]:
### this is just the mass weighted hessian ###
eig = np.linalg.eigvalsh(hess_int) * 4.359744650E-18 / (1.660539040E-27 * 0.529177249E-10 **2)
frequencies = np.sqrt(np.maximum(eig, 0) / (4 * (math.pi ** 2) * (2.99792458e+10 ** 2)))
frequencies

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        , 1292.23926843, 1292.60851439,
       1292.96269545, 1524.43910503, 1524.73700684, 3100.7130439 ,
       3239.15492522, 3240.090936  , 3241.07758875])

In [79]:
eigval, eigvect = np.linalg.eigh(hess_int)
eigval

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.06319422, 0.06323034, 0.063265  , 0.0879451 ,
       0.08797948, 0.36384353, 0.39705891, 0.39728842, 0.39753042])

In [46]:
hess_new = [
    [5.6165400852E-01, 2.0027719437E-06, -3.7522593012E-06, -3.3303966558E-01, -5.8686217902E-05, 2.9415262193E-04, -7.6258791083E-02, 9.0871779419E-02, 2.2688484719E-05, -7.6337720372E-02, -4.5530761030E-02, -7.8781374148E-02, -7.6017831479E-02, -4.5289046736E-02, 7.8468423624E-02],
    [2.0027719437E-06, 5.6158092347E-01, -5.4124387017E-06, -4.3012761166E-05, -4.4038416491E-02, 2.1547610467E-07, 9.0859739481E-02, -3.0088529568E-01, -6.3664938183E-05, -4.5528120343E-02, -1.0835239302E-01, -1.1119019564E-01, -4.5285897356E-02, -1.0830481829E-01, 1.1125900708E-01],
    [-3.7522593012E-06, -5.4124387017E-06, 5.6165278474E-01, 2.9434352971E-04, 2.5598721087E-07, -4.4078260584E-02, 2.2764048094E-05, -6.3724207021E-05, -4.4072614506E-02, -7.8803983642E-02, -1.1120962964E-01, -2.3656251062E-01, 7.8490490001E-02, 1.1127856076E-01, -2.3693939903E-01],
    [-3.3303966558E-01, -4.3012761166E-05, 2.9434352971E-04, 3.5386222593E-01, 5.2500769047E-05, -3.1697024661E-04, -6.9296861066E-03, -2.4308748825E-03, 5.1027229475E-06, -6.9708169818E-03, 1.2147957673E-03, 2.1089453160E-03, -6.9220572597E-03, 1.2104362789E-03, -2.0920773763E-03],
    [-5.8686217902E-05, -4.4038416491E-02, 2.5598721087E-07, 5.2500769047E-05, 4.2747776194E-02, -7.9992806125E-08, 2.9928858854E-02, 2.7584356466E-03, -3.1069934252E-05, -1.4964044756E-02, -7.3347189517E-04, 2.0382635586E-03, -1.4962473820E-02, -7.3432345390E-04, -2.0073609546E-03],
    [2.9415262193E-04, 2.1547610467E-07, -4.4078260584E-02, -3.1697024661E-04, -7.9992806125E-08, 4.2757061907E-02, 2.0483717829E-06, 1.1085705687E-06, -1.8985010721E-03, -2.5904001603E-02, 2.0199157451E-03, 1.6343135323E-03, 2.5925426910E-02, -2.0211684631E-03, 1.5853862172E-03],
    [-7.6258791083E-02, 9.0859739481E-02, 2.2764048094E-05, -6.9296861066E-03, 2.9928858854E-02, 2.0483717829E-06, 7.7378615804E-02, -9.7821593014E-02, -2.4400187738E-05, 2.9168674946E-03, -1.1503941408E-02, 1.2007320081E-03, 2.8929938909E-03, -1.1455902820E-02, -1.2009373278E-03],
    [9.0871779419E-02, -3.0088529568E-01, -6.3724207021E-05, -2.4308748825E-03, 2.7584356466E-03, 1.1085705687E-06, -9.7821593014E-02, 3.1929169028E-01, 6.8754220384E-05, 4.6906028319E-03, -1.0584649827E-02, 2.6390406322E-03, 4.6829245533E-03, -1.0580180425E-02, -2.6458099449E-03],
    [2.2688484719E-05, -6.3664938183E-05, -4.4072614506E-02, 5.1027229475E-06, -3.1069934252E-05, -1.8985010721E-03, -2.4400187738E-05, 6.8754220384E-05, 4.2783013726E-02, 1.0547516240E-02, -2.3758591284E-02, 1.5847053018E-03, -1.0551114172E-02, 2.3785202665E-02, 1.6033965501E-03],
    [-7.6337720372E-02, -4.5528120343E-02, -7.8803983642E-02, -6.9708169818E-03, -1.4964044756E-02, -2.5904001603E-02, 2.9168674946E-03, 4.6906028319E-03, 1.0547516240E-02, 7.7486885692E-02, 4.9021645379E-02, 8.4836731166E-02, 2.9047841669E-03, 6.7767378392E-03, 9.3195238604E-03],
    [-4.5530761030E-02, -1.0835239302E-01, -1.1120962964E-01, 1.2147957673E-03, -7.3347189517E-04, 2.0199157451E-03, -1.1503941408E-02, -1.0584649827E-02, -2.3758591284E-02, 4.9021645379E-02, 1.1196612773E-01, 1.1974030459E-01, 6.8014403404E-03, 7.7043870086E-03, 1.3210546586E-02],
    [-7.8781374148E-02, -1.1119019564E-01, -2.3656251062E-01, 2.1089453160E-03, 2.0382635586E-03, 1.6343135323E-03, 1.2007320081E-03, 2.6390406322E-03, 1.5847053018E-03, 8.4836731166E-02, 1.1974030459E-01, 2.4996147518E-01, -9.3608203630E-03, -1.3229959136E-02, -1.6617983392E-02],
    [-7.6017831479E-02, -4.5285897356E-02, 7.8490490001E-02, -6.9220572597E-03, -1.4962473820E-02, 2.5925426910E-02, 2.8929938909E-03, 4.6829245533E-03, -1.0551114172E-02, 2.9047841669E-03, 6.8014403404E-03, -9.3608203630E-03, 7.7142110681E-02, 4.8760890860E-02, -8.4499457578E-02],
    [-4.5289046736E-02, -1.0830481829E-01, 1.1127856076E-01, 1.2104362789E-03, -7.3432345390E-04, -2.0211684631E-03, -1.1455902820E-02, -1.0580180425E-02, 2.3785202665E-02, 6.7767378392E-03, 7.7043870086E-03, -1.3229959136E-02, 4.8760890860E-02, 1.1191493516E-01, -1.1981450930E-01],
    [7.8468423624E-02, 1.1125900708E-01, -2.3693939903E-01, -2.0920773763E-03, -2.0073609546E-03, 1.5853862172E-03, -1.2009373278E-03, -2.6458099449E-03, 1.6033965501E-03, 9.3195238604E-03, 1.3210546586E-02, -1.6617983392E-02, -8.4499457578E-02, -1.1981450930E-01, 2.5036859965E-01]
]

In [47]:
hess_new_mw = mass_matrix @ hess @ mass_matrix
hess_new_mw

array([[ 0.04738431, -0.00000058,  0.        , -0.09825798,  0.00000259,
         0.        , -0.02175125,  0.02704766,  0.        , -0.02175154,
        -0.01350213, -0.02338653, -0.02175154, -0.01350213,  0.02338653],
       [-0.00000033,  0.04738332,  0.        ,  0.00000086, -0.01221288,
         0.        ,  0.02699909, -0.08869431,  0.        , -0.01349926,
        -0.03129796, -0.03307435, -0.01349926, -0.03129796,  0.03307435],
       [ 0.        ,  0.        ,  0.04738315,  0.        ,  0.        ,
        -0.01221259,  0.        ,  0.        , -0.01221173, -0.0233805 ,
        -0.03306946, -0.06949082,  0.0233805 ,  0.03306946, -0.06949082],
       [-0.0982847 , -0.00000029,  0.        ,  0.36137103, -0.00000595,
         0.        , -0.00743353, -0.0017619 ,  0.        , -0.00743353,
         0.00080655,  0.00139583, -0.00743353,  0.00080655, -0.00139583],
       [ 0.00000115, -0.01219678,  0.        , -0.00000198,  0.04080556,
         0.        ,  0.0305506 ,  0.00278571, 

In [83]:
mass_vec = np.repeat(masses, 3)
inv_sqrt_mass = 1.0 / np.sqrt(mass_vec)
hess_mw = hess * np.outer(inv_sqrt_mass, inv_sqrt_mass)
hess_mw

array([[ 0.04738431, -0.00000058, -0.        , -0.09825798,  0.00000259,
         0.        , -0.02175125,  0.02704766,  0.        , -0.02175154,
        -0.01350213, -0.02338653, -0.02175154, -0.01350213,  0.02338653],
       [-0.00000033,  0.04738332, -0.        ,  0.00000086, -0.01221288,
        -0.        ,  0.02699909, -0.08869431, -0.        , -0.01349926,
        -0.03129796, -0.03307435, -0.01349926, -0.03129796,  0.03307435],
       [-0.        ,  0.        ,  0.04738315,  0.        ,  0.        ,
        -0.01221259, -0.        , -0.        , -0.01221173, -0.0233805 ,
        -0.03306946, -0.06949082,  0.0233805 ,  0.03306946, -0.06949082],
       [-0.0982847 , -0.00000029,  0.        ,  0.36137103, -0.00000595,
         0.        , -0.00743353, -0.0017619 , -0.        , -0.00743353,
         0.00080655,  0.00139583, -0.00743353,  0.00080655, -0.00139583],
       [ 0.00000115, -0.01219678, -0.        , -0.00000198,  0.04080556,
         0.        ,  0.0305506 ,  0.00278571, 

In [116]:
masses_a = np.array(masses)
com = np.sum(geom * masses_a[:, np.newaxis], axis=0) / np.sum(masses_a)
coords = geom - com

# Inertia Tensor & Rotation to Principal Axes
I = np.zeros((3, 3))
for m, r in zip(masses_a, coords):
    I += m * (np.dot(r, r) * np.eye(3) - np.outer(r, r))
_, rot_mat = np.linalg.eigh(I)
coords = np.dot(coords, rot_mat)

# 4. CONSTRUCT PROJECTION VECTORS (D-Matrix)
n_atoms = len(masses_a)
D = np.zeros((6, 3 * n_atoms))

# Translations (X, Y, Z)
for i in range(3):
    D[i, i::3] = np.sqrt(masses_a)

# Rotations (Rx, Ry, Rz)
for i in range(n_atoms):
    x, y, z = coords[i]
    s_m = np.sqrt(masses_a[i])
    D[3, 3*i:3*i+3] = np.array([0, -z, y]) * s_m # Rx
    D[4, 3*i:3*i+3] = np.array([z, 0, -x]) * s_m # Ry
    D[5, 3*i:3*i+3] = np.array([-y, x, 0]) * s_m # Rz

# 5. ORTHOGONALIZE AND PROJECT
# Gram-Schmidt to ensure D vectors are orthonormal
D_norm = []
for v in D:
    for b in D_norm:
        v -= np.dot(v, b) * b
    if np.linalg.norm(v) > 1e-10:
        D_norm.append(v / np.linalg.norm(v))
D_norm = np.array(D_norm)

# Create Projection Matrix P = I - sum(d d^T)
P = np.eye(3 * n_atoms) - np.dot(D_norm.T, D_norm)

# Project Hessian
projected_hess = (P @ hess_mw @ P + (P @ hess_mw @ P).T) / 2

# 6. RESULTS
print("P eigvec:")

print(P)

P eigvec:
[[ 0.25132457  0.          0.         -0.21688733  0.          0.
  -0.21688738  0.         -0.00000004 -0.21688738  0.          0.00000002
  -0.21688738  0.          0.00000002]
 [ 0.          0.25132457  0.          0.         -0.21688733  0.
   0.         -0.21688738  0.          0.         -0.21688738  0.00000003
   0.         -0.21688738 -0.00000003]
 [ 0.          0.          0.25132457  0.          0.         -0.21688737
   0.          0.         -0.21688737  0.          0.         -0.21688737
   0.          0.         -0.21688737]
 [-0.21688733  0.          0.          0.56216903  0.          0.
   0.06216876  0.          0.35355342  0.06216876  0.         -0.17677671
   0.06216876  0.         -0.17677671]
 [ 0.         -0.21688733  0.          0.          0.56216895  0.
   0.          0.06216878  0.          0.          0.06216878 -0.30618625
   0.          0.06216878  0.30618625]
 [ 0.          0.         -0.21688737  0.          0.          0.93716886
   0.        

In [119]:
eig = np.linalg.eigvalsh(hess_mw) * 4.359744650E-18 / (1.660539040E-27 * 0.529177249E-10 **2)
frequencies = np.sqrt(np.maximum(eig, 0) / (4 * (math.pi ** 2) * (2.99792458e+10 ** 2)))
frequencies

array([   0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        , 1292.22553902, 1292.41866597,
       1292.55366488, 1524.37960409, 1524.4862033 , 3100.01978772,
       3239.33923494, 3240.62699567, 3241.12820542])

In [88]:
masses

[12.011, 1.008, 1.008, 1.008, 1.008]